In [3]:
# 라이브러리 불러오기
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import asyncio
import json
import nest_asyncio
import pandas as pd
import re
from tqdm import tqdm
nest_asyncio.apply()

In [2]:
urls = pd.read_csv("../data/KP_ESG_daum.csv")

In [4]:
urls

,Unnamed: 0,0
0,0,https://v.daum.net/v/20181226134507987
1,1,https://v.daum.net/v/20181226181452875
2,2,https://v.daum.net/v/20181227065625371
3,3,https://v.daum.net/v/20181226104242815
4,4,https://v.daum.net/v/20181226110010626
...,...,...
108,108,https://v.daum.net/v/20221014151702626
109,109,https://v.daum.net/v/20221014100438617
110,110,https://v.daum.net/v/20221124145902585
111,111,https://v.daum.net/v/20221124092119025


In [7]:
test = urls.iloc[:, 1:2]

In [10]:
test[:1]

,0
0,https://v.daum.net/v/20181226134507987


In [45]:
# 함수 정의
def get_comments(refer_url, comment_url) : # 댓글 목록을 json 형태로 받아오는 함수
    comments = []
    next = None
    # 처음엔 댓글 개수를 모르므로 충분히 큰 수를 넣어 줌
    comment_count = 10e6
    headers = {
        'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_3) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.0.5 Safari/605.1.15',
        'referer': refer_url
    }
    
    # 수집한 댓글 수가 첫번째에 수집한 총 댓글 수 보다 많다면 반복을 종료합니다.
    while len(comments) < comment_count :
        comment_url_next = comment_url + '&moreParam.next=' + next if next else comment_url
        res = requests.get(comment_url_next, headers=headers)
        dic = json.loads(res.text[res.text.index('(')+1:-2])
        comments.extend(list(map(lambda x : {
            'url' : refer_url,
            'text': x['contents'],
            'reply_count' : x['replyCount'], 
            'uid': x['idNo'],
            'uname' : x['userName'],
            'like': x['sympathyCount'], 
            'dislike': x['antipathyCount'],
            'c_time' : x['modTime'],
            'cid': x['commentNo'],
            'pid' : x['parentCommentNo'] 
            }, dic['result']['commentList'])))
        comment_count = dic['result']['count']['comment']
        next = dic['result']['morePage']['next'] if comment_count else None
    # 필터로 삭제된 댓글을 걸러줍니다
    comments=list(filter(lambda x: len(x['text']), comments))
    return comments



In [46]:
def get_data(oid, aid) :
    try :
        # refer_url: 댓글 보기를 누르면 나오는 댓글 페이지 주소
        # comment_url: 네트워크 탭에서 확인 가능한 동적으로 생성되는 주소
        refer_url = f'https://n.news.naver.com/mnews/article/comment/{oid}/{aid}?sid=100'
        comment_url = f'https://apis.naver.com/commentBox/cbox/web_naver_list_jsonp.json?ticket=news&templateId=default_politics_m3&pool=cbox5&_cv=20220623185938&_callback=jQuery33103200637113167857_1656171100524&lang=ko&country=KR&objectId=news{oid}%2C{aid}&categoryId=&pageSize=100&indexSize=10&groupId=&listType=OBJECT&pageType=more&page=1&initialize=true&userType=&useAltSort=true&replyPageSize=20&followSize=100&sort=new&includeAllStatus=true&_=1656171100525'

        comments = get_comments(refer_url, comment_url)
        comments = pd.DataFrame(comments)
        # 댓글 수가 0개인 기사를 어떻게 할지 추후에 결정해야 함
        return comments
    except :
        return None


In [47]:
df = pd.DataFrame()
for url, title in tqdm(zip(test['0'], test['1'])):
    try:
        oid = re.findall('article/([^\*]*)/', str(url))[0]
        aid = re.findall(f'{oid}/([^\*]*)[?]', str(url))[0]
        sid = url[-3:]
        temp = get_data(oid, aid)
        temp['sid'] = sid
        temp['title'] = title
        df =pd.concat([df, temp])
    except:
        print(url)

969it [03:35,  4.50it/s]


In [48]:
df.reset_index(drop = True, inplace = True)
df

,sid,title,url,text,reply_count,uid,uname,like,dislike,c_time,cid,pid
0,101,여의도 리서치에 불어닥친 ‘30대’ 열풍···증시 위기에 빛난 ‘올드스쿨’ 파워,https://n.news.naver.com/mnews/article/comment...,그러거나 말거나\n공식사기꾼,0.0,MhGy,sunk****,0.0,0.0,2023-01-13T21:00:57+0900,780531340910002577,780531340910002577
1,101,여의도 리서치에 불어닥친 ‘30대’ 열풍···증시 위기에 빛난 ‘올드스쿨’ 파워,https://n.news.naver.com/mnews/article/comment...,주식경력 20년이다.\n애널리스트들 보고서는 주관적인 부분 빼고 객관적인 부분만 참...,0.0,22BNM,dudx****,2.0,0.0,2023-01-13T20:08:32+0900,780527963840970975,780527963840970975
2,101,여의도 리서치에 불어닥친 ‘30대’ 열풍···증시 위기에 빛난 ‘올드스쿨’ 파워,https://n.news.naver.com/mnews/article/comment...,주식과 부동산에 전문가는 없다. 이걸 깨닫는데 수년 걸린다.,0.0,98YkF,saha****,0.0,0.0,2023-01-13T18:20:01+0900,780520972137529356,780520972137529356
3,101,여의도 리서치에 불어닥친 ‘30대’ 열풍···증시 위기에 빛난 ‘올드스쿨’ 파워,https://n.news.naver.com/mnews/article/comment...,애널 삐끼들 공통점 ㅡ어느종목의 의견이 모두 같다. ? 이유는? 목표가만 ...,0.0,3aQw,6003****,0.0,0.0,2023-01-13T18:19:06+0900,780520913534713902,780520913534713902
4,101,여의도 리서치에 불어닥친 ‘30대’ 열풍···증시 위기에 빛난 ‘올드스쿨’ 파워,https://n.news.naver.com/mnews/article/comment...,아직도 애널리스트를 믿는 혹우들이 있나..,0.0,1YwFc,oklo****,0.0,0.0,2023-01-13T17:57:34+0900,780519526058950975,780519526058950975
...,...,...,...,...,...,...,...,...,...,...,...,...
7381,101,2~3위 약진 애널리스트 새 얼굴…증권가에 부는 90년대생 新바람,https://n.news.naver.com/mnews/article/comment...,오동환은 참...아닌거 같다,0.0,2oNWX,akfl****,2.0,0.0,2021-11-23T22:52:12+0900,2692252282,2692252282
7382,101,2~3위 약진 애널리스트 새 얼굴…증권가에 부는 90년대생 新바람,https://n.news.naver.com/mnews/article/comment...,미래에셋대우 김철중애널 리스트님 이름은 틀리지만 신한금융. 나스닥 골드만삭스 파생상...,0.0,5V8As,p946****,0.0,0.0,2021-09-10T03:47:59+0900,2617174032,2617174032
7383,101,2~3위 약진 애널리스트 새 얼굴…증권가에 부는 90년대생 新바람,https://n.news.naver.com/mnews/article/comment...,.,0.0,6n9wD,bund****,1.0,0.0,2020-01-17T11:21:56+0900,1972081432,1972081432
7384,101,2~3위 약진 애널리스트 새 얼굴…증권가에 부는 90년대생 新바람,https://n.news.naver.com/mnews/article/comment...,점쟁이,0.0,3GxCO,4678****,1.0,0.0,2020-01-17T11:06:24+0900,1972063782,1972063782


In [49]:
df.to_pickle('../data/KP_ESG_naver_comments.pkl')
df.to_csv('../data/KP_ESG_naver_comments.csv')